In [ ]:
import os
os.chdir('/')

In [ ]:
# Load the modules
import allel
import zarr
import numcodecs
import numpy as np
import sys
import pandas as pd
import itertools
import ipytree
import glob

In [ ]:
def convert_zarr1kgp(chrom, outfold):
#     chrom = "1"
#     outfold="data"
    # VCF Path
    vcf_path= outfold + '/ALL.chr' + chrom + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
    callset = allel.read_vcf(vcf_path, fields=['numalt'], log=sys.stdout)
    zarr_path = outfold + '/ALL.chr' + chrom + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.zarr'
    print("converting to zarr chrom "+ chrom+ " take a coffee...")
    allel.vcf_to_zarr(vcf_path, zarr_path, group=chrom,fields='*', alt_number=1, log=sys.stdout,
                  overwrite=True)


In [ ]:
def calculate_PBSTaj(chrom, outfold, taj_window):
    zarr_path = outfold + '/ALL.chr' + chrom + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.zarr'
    callset = zarr.open_group(zarr_path, mode='r')
    pos = allel.SortedIndex(callset[ chrom + '/variants/POS'])
    rs = callset[ chrom + '/variants/ID'][:]
    gt_dask = allel.GenotypeDaskArray(callset[chrom + '/calldata/GT'])
    multiall_only = callset[chrom + '/variants/MULTI_ALLELIC'][:]
    SNPs_only = callset[chrom + '/variants/is_snp'][:]
    loc_variants = ~multiall_only & SNPs_only
    pos_forTab = pos[loc_variants]
    rs_forTab = rs[loc_variants]
    pos_forTabv2 = [chrom + "_" + str(s) for s in pos_forTab]
    gt_variant_selection = gt_dask.compress(loc_variants, axis=0)
    panel_path='data/integrated_call_samples_v3.20130502.ALL.panel'
    panel = pd.read_csv(panel_path, sep='\t', usecols=['super_pop', 'pop', 'sample'],encoding='utf-8-sig')
    samples = callset[chrom + '/samples'][:]
    # Test if the samples in the genotype have the same order in the panel loaded
    if np.all(samples == panel['sample'].values):
        samples_list = list(samples)
        samples_callset_index = [samples_list.index(s) for s in panel['sample']]
        panel['callset_index'] = samples_callset_index

        loc_AFR=panel[panel.super_pop == 'AFR'].callset_index.values
        loc_YRI=panel[panel['pop'] == 'YRI'].callset_index.values
        loc_GBR=panel[panel['pop'] == 'GBR'].callset_index.values
        loc_IBS=panel[panel['pop'] == 'IBS'].callset_index.values
        loc_TSI=panel[panel['pop'] == 'TSI'].callset_index.values
        loc_CHB=panel[panel['pop'] == 'CHB'].callset_index.values

        print("converting to gt single pops.. in chrom" + chrom)
        gt_IBS= gt_variant_selection.take(loc_IBS, axis=1).compute()
        gt_TSI= gt_variant_selection.take(loc_TSI, axis=1).compute()
        gt_CHB= gt_variant_selection.take(loc_CHB, axis=1).compute()
        gt_YRI= gt_variant_selection.take(loc_YRI, axis=1).compute()

        print("converting to allele counts... in chrom" + chrom)
        ac_IBS_du = gt_IBS.count_alleles()
        ac_TSI_du = gt_TSI.count_alleles()
        ac_CHB_du = gt_CHB.count_alleles()
        ac_YRI_du = gt_YRI.count_alleles()

        # Make the PBS
        print("Make the PBS of chrom " + chrom + "TSI vs CHB...")
        PBS_TSI_CHB_YRI_2all_noNormed = allel.pbs(ac_TSI_du, ac_CHB_du, ac_YRI_du, window_size=1, normed=False)

        # Make the PBS
        print("Make the PBS of chrom " + chrom + "IBS vs CHB...")
        PBS_IBS_CHB_YRI_2all_noNormed = allel.pbs(ac_IBS_du ,ac_CHB_du, ac_YRI_du, window_size=1, normed=False)

        # Make the PBS
        print("Make the PBS of chrom " + chrom + "CHB vs TSI...")
        PBS_CHB_TSI_YRI_2all_noNormed = allel.pbs(ac_CHB_du, ac_TSI_du, ac_YRI_du, window_size=1, normed=False)

        # Make the PBS
        print("Make the PBS of chrom " + chrom + "CHB vs IBS...")
        PBS_CHB_IBS_YRI_2all_noNormed = allel.pbs(ac_CHB_du, ac_IBS_du, ac_YRI_du, window_size=1, normed=False)
        
        # Make the Tajima
        print("Making the tajima")
        TajD_TSI_YRI_2all = allel.moving_delta_tajima_d(ac_TSI_du, ac_YRI_du, size=taj_window)


        print("Saving the output of chrom "+ chrom)
        # Save the results in a table to be viewed 
        res_df= pd.DataFrame( {'PBS': PBS_TSI_CHB_YRI_2all_noNormed, 'rs' : rs_forTab , 'pos': pos_forTabv2})
        res_df.to_csv('PBS_scikit_'+chrom+ 'TSI' + 'CHB' + 'YRI' +'v2.txt', sep= "\t", na_rep= 'NaN')

        res_df= pd.DataFrame( {'PBS': PBS_IBS_CHB_YRI_2all_noNormed, 'rs' : rs_forTab , 'pos': pos_forTabv2})
        res_df.to_csv('PBS_scikit_'+chrom + 'IBS' + 'CHB' + 'YRI' +'v2.txt', sep= "\t", na_rep= 'NaN')

        res_df= pd.DataFrame( {'PBS': PBS_CHB_TSI_YRI_2all_noNormed, 'rs' : rs_forTab , 'pos': pos_forTabv2})
        res_df.to_csv('PBS_scikit_'+chrom + 'CHB' + 'TSI' + 'YRI' +'v2.txt', sep= "\t", na_rep= 'NaN')

        res_df= pd.DataFrame( {'PBS': PBS_CHB_IBS_YRI_2all_noNormed, 'rs' : rs_forTab , 'pos': pos_forTabv2})
        res_df.to_csv('PBS_scikit_'+chrom + 'CHB' + 'IBS' + 'YRI' +'v2.txt', sep= "\t", na_rep= 'NaN')
        
        # Save Tajima D
        start_every5K = pos_forTab[::taj_window][:-1]
        end_every5K = (pos_forTab[::taj_window][1:]-1)
        
        pos_taj = [str(s) + "_" + str(e) for s,e in zip(start_every5K, end_every5K)]
        
         # Save the TajD Table
        res_df= pd.DataFrame(TajD_TSI_YRI_2all, index= pos_taj, columns= ['TajD'])
        res_df['chromosome'] = chrom
        res_df = res_df.rename_axis("regions").reset_index()

        res_df.to_csv('TajD_scikit_' + chrom + 'TSI' + 'ogYRI' + 'v2.txt', sep= "\t", na_rep= 'NaN')



        print("Job finished, the dinner is on the Tables!!")
    else:
        print("Problem with the samples name! Look the input")

In [ ]:
for c in range(1,23):
    convert_zarr1kgp(str(c), "data")

In [ ]:
for c in range(1,23):
    calculate_PBSTaj(str(c), "data", taj_window= 5000)